In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import keras
from keras import backend as K

cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
cfg.gpu_options.per_process_gpu_memory_fraction=0.333
K.set_session(K.tf.Session(config=cfg))

## CNNs for Computer Vision
## II. Using a pretrained model

#### @author Alec Chapman

This tutorial was adapted from [this keras blog](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)


The data comes from a [Kaggle competition to classify images as being cats or dogs. The data can be downloaded [here](https://www.kaggle.com/c/dogs-vs-cats/data) after signing into Kaggle (either via a Kaggle account or Google, Facebook, or Yahoo!).

### Import some modules needed for our tutorial

In [ ]:
import glob, os
import random
import pickle
import numpy as np
from keras import layers
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.models import load_model
import matplotlib.pyplot as plt

### Set the data directory paths

In [ ]:
# DATADIR = '/home/jovyan/DATA/keras_cat_dog/data'
import getpass
if getpass.getuser() == 'alec':
    DATADIR = "./data_alec/cats_vs_dogs/"
    MODELDIR = './saved_models'
else:
    DATADIR = os.path.join(os.path.expanduser('~'), 'DATA/DeepLearning/data/cats_vs_dogs/')
    MODELDIR = os.path.join(os.path.expanduser('~'), 'DATA/DeepLearning/saved_models')
TRAINDIR = os.path.join(DATADIR, 'train')
VALDIR = os.path.join(DATADIR, 'val')
assert os.path.exists(DATADIR)
assert os.path.exists(MODELDIR)

batch_size = 16

## Overview - Pretrained Models
In the last notebook, we saw how we can create a Convolutional Neural Network from scratch and train it on our data. However, training can take a long time, and you don't want to have to train every time you want to make predictions with your model.

Keras makes it very easy to load in pretrained models. We'll load in a model that was trained using our data and use our validation data to evaluate its performance.

In [ ]:
# !ls /srv/DATA/keras_cat_dog/data

In [ ]:
model = load_model(os.path.join(MODELDIR, 'cats_vs_dogs.h5'))

# Evaluation
We're finally ready to evaluate our model on our validation set! Like in the first notebook, we'll create an image generator that will load images in, reshape them, normalize the pixel values, and pass them along to our model for prediction.

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255) 
validation_generator = test_datagen.flow_from_directory(
        VALDIR,
        target_size=(227, 227),
        batch_size=batch_size,
        class_mode='binary')

In [ ]:
loss, acc = model.evaluate_generator(validation_generator, steps=800//batch_size, verbose=1)
print("Accuracy: {} Loss: {}".format(acc, loss))

Let's visualize our predictions. The left column will show all of the images that our model thinks are cats, the right will show all of the images that our model thinks are dogs


In [ ]:
x_batch, y_batch = next(validation_generator)
y_pred = model.predict_classes(x_batch)

In [ ]:
ncols = 2
nimg = x_batch.shape[0]

fig = plt.figure(figsize=(18,9))
for i in range(len(x_batch)):
    x = x_batch[i]
    y = y_batch[i]
    ax = plt.subplot2grid((nimg//ncols+1, ncols), (i//ncols,y_pred[i][0]))
    ax.imshow(x)
    #img = array_to_img(x)
    #img.show()

Now let's look at the training and validation accuracies at each stage in training:

In [ ]:
# Load the training and validation accuracy history
with open('logs/history_cats_vs_dogs.pkl', 'rb') as f:
    h = pickle.load(f)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(h['acc'], marker='.', linestyle='dotted', alpha=0.4, label='Cats Vs. Dogs Train Acc')
plt.plot(h['val_acc'], marker='.', label="Cats Vs. Dogs Val Acc")
plt.xlabel('# epochs')
plt.ylim((0.5, 0.92))
plt.legend(loc='upper center', ncol=2,mode='expand')

# Keras's Pretrained Models:
Keras has a number of models available that have been trained with ImageNet data, an image recognition challenge that has 1000 classes and large, large training datasets. We can quickly download one of those models and see what it does on a few of our images:


In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

In [ ]:
model = ResNet50(weights='imagenet')

In [ ]:
model.summary()

In [ ]:
# Pick a cat and dog image at random
cat = random.choice(glob.glob(os.path.join(TRAINDIR, 'cat', 'cat*')))
dog = random.choice(glob.glob(os.path.join(TRAINDIR, 'dog', 'dog*')))

In [ ]:
img_cat = image.load_img(cat, target_size=(224, 224))
img_cat

In [ ]:
x_cat = image.img_to_array(img_cat)
x_cat = np.expand_dims(x_cat, axis=0)
x_cat = preprocess_input(x_cat)

preds = model.predict(x_cat)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

In [ ]:
img_dog = image.load_img(dog, target_size=(224, 224))
img_dog

In [ ]:
x_dog = image.img_to_array(img_dog)
x_dog = np.expand_dims(x_dog, axis=0)
x_dog = preprocess_input(x_dog)

preds = model.predict(x_dog)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

# Visualizing Intermediate Layers
Let's look at how this model is functioning.

**Note** - These examples are taken from [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python)

In [ ]:
from keras import models

# Extracts every other models for the first 30 layers, plus two of the later layers:
# For some reason this started throwing an error when predicting the first layer
layer_outputs = [layer.output for (i, layer) in enumerate(model.layers[1:30]) if i % 2 == 0]

# Creates a model that will return these outputs, given the model input:
activation_model = models.Model(input=model.input, output=layer_outputs)

In [ ]:
layer_outputs

In [ ]:
from random import choice

In [ ]:
def plot_intermediate_layers(img_tensor):
    activations = activation_model.predict(img_tensor)
    for activation_layer in activations:
        _,_,_, channels = activation_layer.shape
        # Pick a random channel to plot
        channel_to_plot = random.choice(range(channels))
        plt.matshow(activation_layer[0, :, :, channel_to_plot], cmap='viridis')

In [ ]:
plot_intermediate_layers(x_cat)

In [ ]:
plot_intermediate_layers(x_dog)

## Conclusion
We can use pretrained models to easily predict using models that were already trained. These models can be made from scratch or be one of a number of pre-existing models. This can allow us to utilize heavy computing resources for training and then be able to predict much more easily.